### Начало работы

In [1]:
# Подключаем библиотеки numpy, pandas и math (для числа ПИ)

import numpy as np
import pandas as pd
import math

In [2]:
# Считываем csv файл с GitHub

df_planets = pd.read_csv(r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Planets.csv")
df_planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [3]:
df_planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


### Написать функцию и применить её к сгруппированным данным с помощью .apply()

##### Не могу сказать точно в каких единицах представлены значения в столбцах, но для расчета предположим, что скорее всего orbital_period - в днях, distance - в млн. км. Посчитаем среднюю массу звезд для планет, сгруппированных по Методу открытия.
##### Формула для расчета массы звезды: $M=\frac{R^3 4 \pi^2}{T^2 G}$, где R - расстояние от планеты до звезды (в метрах), T - орбитальный период в секундах, G - гравитационная постоянная

In [4]:
def get_mass_stars(df):
    G = 6.6743 * 10**(-11)     # Это гравитационная постоянная
    R = df['distance'].mean() * 10**(9)     # Берем среднюю дистанцию и переводим дистанцию из млн км в метры
    T = df['orbital_period'].mean() * 24 * 60 * 60     # Берем средний орбитальный период и переводим орбитальный период из дней в секунды
    M = (R**(3) * 4 * math.pi**(2)) / (T**(2) * G)     # Считаем среднюю массу по формуле, измеряется в килограммах
    return M

In [5]:
(
    df_planets
    .groupby('method')     # Группируем строки по столбцу Method
    .apply(get_mass_stars, include_groups=False)
    .reset_index(drop = False)
    .rename(columns={0: 'Средняя масса звезд'})
)

,method,Средняя масса звезд
0,Astrometry,1.135947e+27
1,Eclipse Timing Variations,1.100673e+29
2,Imaging,1.759598e+24
3,Microlensing,5.669966e+32
4,Orbital Brightness Modulation,2.587644e+38
5,Pulsar Timing,2.539339e+30
6,Pulsation Timing Variations,NaN
7,Radial Velocity,1.605857e+28
8,Transit,3.830055e+34
9,Transit Timing Variations,1.676490e+34


### Написать функцию и применить её к сгрупированным данным с помощью .transform()

In [7]:
# Найдем максимальный орбитальный период из группировки

def orbital_period_max(df):
    return df.max()

In [30]:
# Вызовем метод трансформ, чтобы найти максимальный орбитальный период по группам

df_planets['orbital_period_max'] = df_planets.groupby('method')['orbital_period'].transform(orbital_period_max)
df_planets

,method,number,orbital_period,mass,distance,year,orbital_period_max
0,Radial Velocity,1,269.300000,7.10,77.40,2006,17337.50000
1,Radial Velocity,1,874.774000,2.21,56.95,2008,17337.50000
2,Radial Velocity,1,763.000000,2.60,19.84,2011,17337.50000
3,Radial Velocity,1,326.030000,19.40,110.62,2007,17337.50000
4,Radial Velocity,1,516.220000,10.50,119.47,2009,17337.50000
...,...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006,331.60059
1031,Transit,1,2.615864,NaN,148.00,2007,331.60059
1032,Transit,1,3.191524,NaN,174.00,2007,331.60059
1033,Transit,1,4.125083,NaN,293.00,2008,331.60059


##### Далее хочется посчитать, а сколько оборотов вокруг своей звезды делает каждая планета за время 1 оборота планеты, орбитальный период которой является максимальным в группе

In [25]:
# Напишем функцию, которая считает количество полных оборотов для всех планет относиетльно 1 оборота планеты с максимальным орбитальным периодом

def number_of_revolutions(df):
    df['number_of_revolutions'] = df['orbital_period_max'] // df['orbital_period']
    return df

In [29]:
# Применим функцию к сгруппированным данным по Методу открытия

(
    df_planets
    .groupby('method')
    .apply(number_of_revolutions, include_groups=False)
    .head(50)
    #.reset_index(drop = False)
    #.drop('level_1', axis = 1)
)

number  orbital_period  mass  distance  year  \
method                                                                         
Astrometry                113        1          246.36   NaN     20.77  2013   
                          537        1         1016.00   NaN     14.98  2010   
Eclipse Timing Variations 32         1        10220.00  6.05       NaN  2009   
                          37         2         5767.00   NaN    130.72  2008   
                          38         2         3321.00   NaN    130.72  2008   
                          39         2         5573.55   NaN    500.00  2010   
                          40         2         2883.50   NaN    500.00  2010   
                          41         1         2900.00   NaN       NaN  2011   
                          42         1         4343.50  4.20       NaN  2012   
                          43         2         5840.00   NaN       NaN  2011   
                          44         2         1916.25   NaN       NaN  2011   
Imaging                   29         1             NaN   NaN     45.52  2005   
                          30         1             NaN   NaN    165.00  2007   
                          31         1             NaN   NaN    140.00  2004   
                          33         1             NaN   NaN       NaN  2008   
                          34         1             NaN   NaN    145.00  2013   
                          35         1             NaN   NaN    139.00  2004   
                          36         1             NaN   NaN     18.39  2006   
                          47         1         6000.00   NaN     19.28  2008   
                          54         1             NaN   NaN     52.03  2012   
                          68         1       318280.00   NaN      7.69  2008   
                          69         1             NaN   NaN    145.00  2008   
                          70         1             NaN   NaN     36.00  2013   
                          71         1             NaN   NaN    140.00  2010   
                          72         1         4639.15   NaN     12.21  2009   
                          73         1             NaN   NaN     52.40  2004   
                          74         1         7336.50   NaN     25.00  2009   
                          75         1         8679.70   NaN     26.67  2009   
                          89         1        10037.50   NaN     23.10  2011   
                          90         1             NaN   NaN       NaN  2006   
                          124        1             NaN   NaN     17.95  2013   
                          150        1             NaN   NaN    145.00  2010   
                          392        1             NaN   NaN     91.57  2013   
                          423        1             NaN   NaN     91.83  2013   
                          577        1             NaN   NaN     40.85  2006   
                          635        1             NaN   NaN     11.43  2010   
                          639        1             NaN   NaN    131.93  2010   
                          643        4       170000.00   NaN     39.94  2008   
                          644        4        69000.00   NaN     39.94  2008   
                          645        4        37000.00   NaN     39.94  2008   
                          646        4        18000.00   NaN     39.94  2010   
                          900        1        40000.00   NaN       NaN  2011   
                          937        1       730000.00   NaN       NaN  2006   
                          948        1             NaN   NaN    135.00  2013   
                          949        1             NaN   NaN    120.00  2013   
                          950        1             NaN   NaN       NaN  2010   
                          957        1             NaN   NaN       NaN  2008   
                          1027       1             NaN   NaN     19.20  2011   
                          1029  

### Написать функцию и применить её ко всем данным с помощью .map()

In [6]:
# Сравним дистанцию планет от своих звезд с расстоянием Земли от Солнца (примерно 150 млн.км.)

a_chto_dalshe = lambda x: 'Ближе к своей звезде, чем Земля к Солнцу' if x <= 150 else 'Дальше от своей звезды, чем Земля от Солнца'

In [19]:
# Добавим столбец с аналитикой в исходный датафрэйм

df_planets['dist_analitika'] = (
    df_planets['distance']
    .map(a_chto_dalshe)
    #.reset_index(drop = False)
    #.rename(columns={0: 'Дальше/Ближе'})
    #.drop('level_0', axis = 1)
    #.drop('index', axis = 1)
)
df_planets

,method,number,orbital_period,mass,distance,year,dist_analitika
0,Radial Velocity,1,269.300000,7.10,77.40,2006,"Ближе к своей звезде, чем Земля к Солнцу"
1,Radial Velocity,1,874.774000,2.21,56.95,2008,"Ближе к своей звезде, чем Земля к Солнцу"
2,Radial Velocity,1,763.000000,2.60,19.84,2011,"Ближе к своей звезде, чем Земля к Солнцу"
3,Radial Velocity,1,326.030000,19.40,110.62,2007,"Ближе к своей звезде, чем Земля к Солнцу"
4,Radial Velocity,1,516.220000,10.50,119.47,2009,"Ближе к своей звезде, чем Земля к Солнцу"
...,...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006,"Дальше от своей звезды, чем Земля от Солнца"
1031,Transit,1,2.615864,NaN,148.00,2007,"Ближе к своей звезде, чем Земля к Солнцу"
1032,Transit,1,3.191524,NaN,174.00,2007,"Дальше от своей звезды, чем Земля от Солнца"
1033,Transit,1,4.125083,NaN,293.00,2008,"Дальше от своей звезды, чем Земля от Солнца"


### Сгруппировать данные по дате (если применимо) и использовать resample для агрегирования в более крупный период. Если нет дат то испольовать БД dvdrental для выполнения этого пункта.